# pycsp3-scheduling: API Reference & Interactive Tutorials

This notebook provides comprehensive documentation and interactive examples for the pycsp3-scheduling library.

**pycsp3-scheduling** extends [pycsp3](https://pycsp.org) with scheduling support including:
- Interval variables for tasks/activities
- Sequence variables for machine scheduling
- Precedence and grouping constraints
- Cumulative functions for resource modeling
- State functions for discrete resource states

## Table of Contents

1. [Import and Setup](#1-import-and-setup)
2. [IntervalVar API Reference](#2-intervalvar-api-reference)
3. [SequenceVar API Reference](#3-sequencevar-api-reference)
4. [Precedence Constraints API](#4-precedence-constraints-api)
5. [Grouping Constraints API](#5-grouping-constraints-api)
6. [Cumulative Functions API](#6-cumulative-functions-api)
7. [State Functions API](#7-state-functions-api)
8. [Tutorial: Basic Job Shop Modeling](#8-tutorial-basic-job-shop-modeling)
9. [Tutorial: Flexible Job Shop](#9-tutorial-flexible-job-shop)
10. [Tutorial: RCPSP](#10-tutorial-rcpsp)
11. [Tutorial: Sequences and NoOverlap](#11-tutorial-sequences-and-nooverlap)
12. [Tutorial: Cumulative Resources](#12-tutorial-cumulative-resources)
13. [Tutorial: State Functions](#13-tutorial-state-functions)

## 1. Import and Setup

Import the core modules from pycsp3-scheduling and pycsp3:

In [ ]:
# Core pycsp3 imports
from pycsp3 import satisfy, minimize, solve, SAT, OPTIMUM, Maximum, Sum, Var

# pycsp3-scheduling: Variables
from pycsp3_scheduling import (
    IntervalVar, IntervalVarArray, IntervalVarDict,
    SequenceVar, SequenceVarArray,
    INTERVAL_MIN, INTERVAL_MAX,
)

# pycsp3-scheduling: Expressions
from pycsp3_scheduling import (
    start_of, end_of, size_of, length_of, presence_of,
    overlap_length, expr_min, expr_max,
)

# pycsp3-scheduling: Interop (for pycsp3 integration)
from pycsp3_scheduling import start_time, end_time, presence_time, interval_value

# pycsp3-scheduling: Constraints
from pycsp3_scheduling import (
    # Precedence
    end_before_start, start_before_start, end_before_end, start_before_end,
    start_at_start, start_at_end, end_at_start, end_at_end,
    # Grouping
    span, alternative, synchronize,
    # Sequence
    SeqNoOverlap, first, last, before, previous,
    same_sequence, same_common_subsequence,
)

# pycsp3-scheduling: Cumulative Functions
from pycsp3_scheduling import (
    CumulFunction, pulse, step_at, step_at_start, step_at_end,
    cumul_range, always_in, height_at_start, height_at_end,
)

# pycsp3-scheduling: State Functions
from pycsp3_scheduling import (
    StateFunction, TransitionMatrix,
    always_equal, always_constant, always_no_state,
)

print("✓ All imports successful!")
print(f"pycsp3-scheduling loaded")

## 2. IntervalVar API Reference

### Creating Interval Variables

An `IntervalVar` represents a task or activity with:
- **start**: Start time bounds `(min, max)` or fixed `int`
- **end**: End time bounds `(min, max)` or fixed `int`
- **size**: Duration bounds `(min, max)` or fixed `int`
- **length**: Length bounds (can differ from size with intensity functions)
- **optional**: If `True`, the interval may be absent
- **name**: Unique identifier

In [ ]:
# Reset registry for clean examples
from pycsp3_scheduling.variables.interval import clear_interval_registry
clear_interval_registry()

# Fixed duration task
task1 = IntervalVar(size=10, name="task1")
print(f"task1: size=[{task1.size_min}, {task1.size_max}], is_fixed_size={task1.is_fixed_size}")

# Variable duration task
task2 = IntervalVar(size=(5, 20), name="task2")
print(f"task2: size=[{task2.size_min}, {task2.size_max}], is_fixed_size={task2.is_fixed_size}")

# Task with bounded start and end
task3 = IntervalVar(start=(0, 100), end=(10, 200), size=10, name="task3")
print(f"task3: start=[{task3.start_min}, {task3.start_max}], end=[{task3.end_min}, {task3.end_max}]")

# Optional task (can be absent from solution)
task4 = IntervalVar(size=15, optional=True, name="optional_task")
print(f"task4: optional={task4.is_optional}, is_present={task4.is_present}")

### IntervalVarArray and IntervalVarDict

Factory functions for creating collections of interval variables:

In [ ]:
# Create 1D array of intervals
tasks = IntervalVarArray(5, size_range=10, name="task")
print(f"Created {len(tasks)} tasks: {[t.name for t in tasks]}")

# Create 2D array (jobs × operations)
ops = IntervalVarArray((3, 4), size_range=(5, 15), optional=True, name="op")
print(f"2D array shape: {len(ops)} x {len(ops[0])}")
print(f"ops[1][2].name = {ops[1][2].name}")

# Create dictionary of intervals
stages = IntervalVarDict(
    keys=["assembly", "testing", "packaging"],
    size_range=20,
    name="stage"
)
print(f"Stage keys: {list(stages.keys())}")
print(f"stages['testing'].size_min = {stages['testing'].size_min}")

## 3. SequenceVar API Reference

A `SequenceVar` represents an ordered sequence of intervals on a disjunctive resource (e.g., a machine):

In [ ]:
clear_interval_registry()

# Create tasks for a machine
machine_tasks = [IntervalVar(size=10, name=f"m_task{i}") for i in range(4)]

# Simple sequence (no types)
machine = SequenceVar(intervals=machine_tasks, name="machine1")
print(f"Sequence '{machine.name}' has {len(machine)} intervals")
print(f"has_types: {machine.has_types}")

# Sequence with types (for transition matrices)
job_types = [0, 1, 0, 1]  # Type per task
typed_machine = SequenceVar(
    intervals=machine_tasks,
    types=job_types,
    name="machine_with_types"
)
print(f"\nTyped sequence: has_types={typed_machine.has_types}")
print(f"Task 2 type: {typed_machine.get_type(2)}")
print(f"Tasks of type 0: {[t.name for t in typed_machine.get_intervals_by_type(0)]}")

## 4. Precedence Constraints API

Precedence constraints define temporal relationships between intervals.

### Before Constraints (Inequalities)

| Constraint | Semantics |
|------------|-----------|
| `end_before_start(a, b, delay=0)` | `start(b) >= end(a) + delay` |
| `start_before_start(a, b, delay=0)` | `start(b) >= start(a) + delay` |
| `end_before_end(a, b, delay=0)` | `end(b) >= end(a) + delay` |
| `start_before_end(a, b, delay=0)` | `end(b) >= start(a) + delay` |

### At Constraints (Equalities)

| Constraint | Semantics |
|------------|-----------|
| `start_at_start(a, b, delay=0)` | `start(b) == start(a) + delay` |
| `start_at_end(a, b, delay=0)` | `start(b) == end(a) + delay` |
| `end_at_start(a, b, delay=0)` | `end(a) == start(b) + delay` |
| `end_at_end(a, b, delay=0)` | `end(b) == end(a) + delay` |

In [ ]:
clear_interval_registry()

# Create example tasks
task_a = IntervalVar(size=10, name="A")
task_b = IntervalVar(size=15, name="B")

# Basic precedence: A must finish before B starts
constraint1 = end_before_start(task_a, task_b)
print(f"end_before_start(A, B): {constraint1}")

# With delay: A must finish at least 5 units before B starts
constraint2 = end_before_start(task_a, task_b, delay=5)
print(f"end_before_start(A, B, delay=5): {constraint2}")

# Synchronization: A and B start together
constraint3 = start_at_start(task_a, task_b)
print(f"start_at_start(A, B): {constraint3}")

# Chain: A ends exactly when B starts (no gap)
constraint4 = start_at_end(task_a, task_b)
print(f"start_at_end(A, B): B starts when A ends")

## 5. Grouping Constraints API

### span(main, subtasks)
Main interval spans all present subtasks (starts at earliest, ends at latest).

### alternative(main, alternatives, cardinality=1)
Select exactly `cardinality` alternatives that match the main interval.

### synchronize(main, intervals)
All present intervals synchronize with main interval (same start/end).

In [ ]:
clear_interval_registry()

# Example: span constraint
# Main task spans all its subtasks
main_task = IntervalVar(name="project")
phase1 = IntervalVar(size=10, name="phase1")
phase2 = IntervalVar(size=15, name="phase2")
phase3 = IntervalVar(size=8, name="phase3")

span_constraint = span(main_task, [phase1, phase2, phase3])
print(f"span(main, [phase1, phase2, phase3]):")
print(f"  -> {span_constraint}")

# Example: alternative constraint
# Select exactly 1 machine to process the task
task = IntervalVar(size=10, name="task")
on_machine1 = IntervalVar(size=10, optional=True, name="on_m1")
on_machine2 = IntervalVar(size=12, optional=True, name="on_m2")
on_machine3 = IntervalVar(size=8, optional=True, name="on_m3")

alt_constraint = alternative(task, [on_machine1, on_machine2, on_machine3])
print(f"\nalternative(task, [on_m1, on_m2, on_m3]):")
print(f"  -> {alt_constraint}")

# Example: synchronize constraint
# All subtasks must match the main interval timing
meeting = IntervalVar(size=60, name="meeting")
attendee1 = IntervalVar(optional=True, name="attendee1")
attendee2 = IntervalVar(optional=True, name="attendee2")

sync_constraint = synchronize(meeting, [attendee1, attendee2])
print(f"\nsynchronize(meeting, [attendee1, attendee2]):")
print(f"  -> {sync_constraint}")

## 6. Cumulative Functions API

Cumulative functions model resource usage over time.

| Function | Description |
|----------|-------------|
| `pulse(interval, height)` | Resource used during interval execution |
| `step_at_start(interval, height)` | Step increase at interval start |
| `step_at_end(interval, height)` | Step increase at interval end |
| `step(time, height)` | Step increase at fixed time point |
| `cumul_range(func, min, max)` | Constrain cumulative function range |

In [ ]:
clear_interval_registry()

# Example: Cumulative resource with capacity constraint
task1 = IntervalVar(size=10, name="task1")
task2 = IntervalVar(size=15, name="task2")
task3 = IntervalVar(size=8, name="task3")

# Each task uses some capacity during execution
p1 = pulse(task1, 3)  # Uses 3 units
p2 = pulse(task2, 2)  # Uses 2 units
p3 = pulse(task3, 4)  # Uses 4 units

print(f"pulse(task1, 3) -> {p1}")
print(f"pulse(task2, 2) -> {p2}")
print(f"pulse(task3, 4) -> {p3}")

# Combine into cumulative function and constrain
cumul = p1 + p2 + p3
capacity_constraint = cumul_range(cumul, 0, 5)  # Max capacity of 5
print(f"\ncumul_range(p1 + p2 + p3, 0, 5) -> {capacity_constraint}")

# Example: Step functions for inventory/state changes
incoming = IntervalVar(size=5, name="delivery")
outgoing = IntervalVar(size=3, name="shipment")

# Inventory increases at delivery end, decreases at shipment start
inv_up = step_at_end(incoming, 10)   # +10 units delivered
inv_down = step_at_start(outgoing, -5)  # -5 units shipped

print(f"\nstep_at_end(delivery, 10) -> {inv_up}")
print(f"step_at_start(shipment, -5) -> {inv_down}")

## 7. State Functions API

State functions model resource states that change over time (e.g., machine configurations).

| Function | Description |
|----------|-------------|
| `StateFunction(name)` | Create a state function |
| `TransitionMatrix(n, default)` | Create n×n transition matrix |
| `always_equal(func, interval, state)` | State equals value during interval |
| `always_constant(func, interval)` | State constant during interval |

In [ ]:
clear_interval_registry()

# Example: Machine with different configurations (states)
# States: 0=idle, 1=setup_A, 2=setup_B, 3=processing

# Create state function for machine configuration
machine_state = StateFunction("machine")
print(f"StateFunction: {machine_state}")

# Transition matrix: time to change between states
# transitions[from_state][to_state] = time required
transitions = TransitionMatrix(4, default=0)
transitions[0, 1] = 5   # idle -> setup_A: 5 time units
transitions[0, 2] = 5   # idle -> setup_B: 5 time units
transitions[1, 2] = 10  # setup_A -> setup_B: 10 time units
transitions[2, 1] = 10  # setup_B -> setup_A: 10 time units
print(f"\nTransitionMatrix:\n{transitions}")

# Tasks require specific machine states
task_a1 = IntervalVar(size=20, name="task_A1")
task_a2 = IntervalVar(size=15, name="task_A2")
task_b1 = IntervalVar(size=25, name="task_B1")

# Task A1 and A2 require setup_A (state 1)
state_a1 = always_equal(machine_state, task_a1, 1)
state_a2 = always_equal(machine_state, task_a2, 1)

# Task B1 requires setup_B (state 2)
state_b1 = always_equal(machine_state, task_b1, 2)

print(f"\nalways_equal(machine, task_A1, 1) -> {state_a1}")
print(f"always_equal(machine, task_B1, 2) -> {state_b1}")

---

# Part 2: Interactive Tutorials

Now let's apply these concepts to solve real scheduling problems!

## Tutorial 1: Simple Job Shop Scheduling

**Problem**: Schedule 2 jobs on 2 machines. Each job has 2 operations that must run in order. Each machine can process one operation at a time.

```
Job 1: Machine1(3) → Machine2(2)
Job 2: Machine2(4) → Machine1(2)
```

**Goal**: Minimize makespan (total completion time).

In [ ]:
clear_interval_registry()

# Problem data
# jobs[job_id] = [(machine_id, processing_time), ...]
jobs = {
    0: [(0, 3), (1, 2)],  # Job 0: Machine0 for 3, then Machine1 for 2
    1: [(1, 4), (0, 2)],  # Job 1: Machine1 for 4, then Machine0 for 2
}

n_jobs = len(jobs)
n_machines = 2

# Create interval variables for each operation
# operations[job][op_index] = IntervalVar
operations = {}
for job, ops in jobs.items():
    operations[job] = []
    for op_idx, (machine, duration) in enumerate(ops):
        iv = IntervalVar(
            size=duration,
            name=f"job{job}_op{op_idx}_m{machine}"
        )
        operations[job].append((iv, machine))
        
print("Created interval variables:")
for job, ops in operations.items():
    for iv, machine in ops:
        print(f"  {iv.name}: size={iv.size_min} on machine {machine}")

In [ ]:
# Add precedence constraints: operations within a job must run in order
precedence_constraints = []

for job, ops in operations.items():
    for i in range(len(ops) - 1):
        iv_before, _ = ops[i]
        iv_after, _ = ops[i + 1]
        constraint = end_before_start(iv_before, iv_after)
        precedence_constraints.append(constraint)
        print(f"Precedence: {iv_before.name} -> {iv_after.name}")

print(f"\nTotal precedence constraints: {len(precedence_constraints)}")

In [ ]:
# Add machine constraints: no two operations on same machine can overlap
# Group operations by machine
machine_ops = {m: [] for m in range(n_machines)}
for job, ops in operations.items():
    for iv, machine in ops:
        machine_ops[machine].append(iv)

# Create sequence variables for each machine
sequences = {}
for machine, ops in machine_ops.items():
    seq = SequenceVar(intervals=ops, name=f"machine{machine}")
    sequences[machine] = seq
    print(f"Machine {machine} sequence: {[iv.name for iv in ops]}")

# No overlap constraint on each machine
no_overlap_constraints = []
for machine, seq in sequences.items():
    constraint = SeqNoOverlap(seq)
    no_overlap_constraints.append(constraint)
    print(f"NoOverlap on machine {machine}: {constraint}")

print(f"\nTotal machine constraints: {len(no_overlap_constraints)}")

In [ ]:
# Summary of the Job Shop model
print("=" * 50)
print("Job Shop Scheduling Model Summary")
print("=" * 50)
print(f"\nVariables:")
print(f"  - {sum(len(ops) for ops in operations.values())} interval variables (operations)")
print(f"  - {len(sequences)} sequence variables (machines)")
print(f"\nConstraints:")
print(f"  - {len(precedence_constraints)} precedence constraints")
print(f"  - {len(no_overlap_constraints)} no-overlap constraints")
print(f"\nObjective: Minimize makespan (max end time of all operations)")

# To solve this, you would:
# 1. Post all constraints to pycsp3
# 2. Add makespan variable and minimize it
# 3. Call a constraint solver

## Tutorial 2: Resource-Constrained Project Scheduling (RCPSP)

**Problem**: Schedule project activities with:
- Precedence relations between activities
- Limited renewable resource capacity

```
Activities: A(5), B(3), C(4), D(6)
Precedence: A → B, A → C, B → D, C → D
Resource capacity: 3 units
Resource usage: A=2, B=1, C=2, D=1
```

In [ ]:
clear_interval_registry()

# Activity data: (duration, resource_usage)
activities = {
    'A': (5, 2),
    'B': (3, 1),
    'C': (4, 2),
    'D': (6, 1),
}

# Precedence relations
precedences = [
    ('A', 'B'),
    ('A', 'C'),
    ('B', 'D'),
    ('C', 'D'),
]

# Resource capacity
resource_capacity = 3

# Create interval variables
tasks = {}
for name, (duration, _) in activities.items():
    tasks[name] = IntervalVar(size=duration, name=name)
    print(f"Activity {name}: duration={duration}")

print(f"\nCreated {len(tasks)} activity intervals")

In [ ]:
# Add precedence constraints
prec_constraints = []
for before, after in precedences:
    constraint = end_before_start(tasks[before], tasks[after])
    prec_constraints.append(constraint)
    print(f"Precedence: {before} → {after}")

# Add resource constraint using cumulative function
resource_usage = None
for name, (_, usage) in activities.items():
    p = pulse(tasks[name], usage)
    if resource_usage is None:
        resource_usage = p
    else:
        resource_usage = resource_usage + p
    print(f"Activity {name} uses {usage} units")

# Constrain cumulative resource usage
resource_constraint = cumul_range(resource_usage, 0, resource_capacity)
print(f"\nResource capacity constraint: 0 <= usage <= {resource_capacity}")
print(f"Constraint: {resource_constraint}")

## Tutorial 3: Flexible Job Shop with Alternative Machines

**Problem**: Jobs can be processed on alternative machines with different processing times.

```
Job 1, Op 1: Machine1(5) OR Machine2(4)
Job 1, Op 2: Machine2(3) OR Machine3(2)
```

**Key concept**: Use `alternative()` to model machine choice.

In [ ]:
clear_interval_registry()

# Flexible operation data
# operation = [(machine_id, duration), ...]
flexible_ops = {
    ('job1', 0): [(0, 5), (1, 4)],           # Op1: M1(5) or M2(4)
    ('job1', 1): [(1, 3), (2, 2)],           # Op2: M2(3) or M3(2)
    ('job2', 0): [(0, 3), (2, 4)],           # Op1: M1(3) or M3(4)
    ('job2', 1): [(0, 2), (1, 3), (2, 2)],   # Op2: any machine
}

n_machines = 3

# Create main operation intervals (abstract timing)
main_ops = {}
for (job, op_idx), alternatives in flexible_ops.items():
    # Main interval: variable size based on machine choice
    min_dur = min(dur for _, dur in alternatives)
    max_dur = max(dur for _, dur in alternatives)
    main_ops[(job, op_idx)] = IntervalVar(
        size=(min_dur, max_dur),
        name=f"{job}_op{op_idx}"
    )
    print(f"{job}_op{op_idx}: size in [{min_dur}, {max_dur}]")

print(f"\nCreated {len(main_ops)} main operation intervals")

In [ ]:
# Create alternative intervals for each machine choice
machine_intervals = {m: [] for m in range(n_machines)}
alternative_constraints = []

for (job, op_idx), alternatives in flexible_ops.items():
    main = main_ops[(job, op_idx)]
    alt_intervals = []
    
    for machine, duration in alternatives:
        alt_iv = IntervalVar(
            size=duration,
            optional=True,  # May or may not be selected
            name=f"{job}_op{op_idx}_m{machine}"
        )
        alt_intervals.append(alt_iv)
        machine_intervals[machine].append(alt_iv)
    
    # Exactly one alternative must be selected
    alt_constraint = alternative(main, alt_intervals)
    alternative_constraints.append(alt_constraint)
    
    print(f"{main.name}: alternatives = {[iv.name for iv in alt_intervals]}")

print(f"\nCreated {len(alternative_constraints)} alternative constraints")

In [ ]:
# No overlap on each machine (among selected alternatives)
machine_sequences = {}
for machine, intervals in machine_intervals.items():
    if intervals:
        seq = SequenceVar(intervals=intervals, name=f"machine{machine}")
        machine_sequences[machine] = seq
        no_overlap = SeqNoOverlap(seq)
        print(f"Machine {machine}: {len(intervals)} optional operations")

# Job precedence: operations within each job must be ordered
for job in ['job1', 'job2']:
    ops_in_job = sorted([k for k in main_ops.keys() if k[0] == job])
    for i in range(len(ops_in_job) - 1):
        before = main_ops[ops_in_job[i]]
        after = main_ops[ops_in_job[i + 1]]
        prec = end_before_start(before, after)
        print(f"Precedence: {before.name} → {after.name}")

## Tutorial 4: Sequence-Dependent Setup Times

**Problem**: Machines require setup time when switching between different product types.

**Key concept**: Use `SeqNoOverlap()` with a transition matrix.

In [ ]:
clear_interval_registry()

# Products have types: 0=TypeA, 1=TypeB, 2=TypeC
# Setup times depend on switching between types
n_types = 3

# Setup time matrix: setup[from_type][to_type]
setup_times = TransitionMatrix(n_types, default=2)
# Same type: no setup needed
setup_times[0, 0] = 0
setup_times[1, 1] = 0
setup_times[2, 2] = 0
# Different types: various setup times
setup_times[0, 1] = 5  # TypeA -> TypeB
setup_times[1, 0] = 4  # TypeB -> TypeA
setup_times[0, 2] = 3  # TypeA -> TypeC
setup_times[2, 0] = 3  # TypeC -> TypeA
setup_times[1, 2] = 6  # TypeB -> TypeC
setup_times[2, 1] = 6  # TypeC -> TypeB

print("Setup time matrix:")
print(setup_times)

In [ ]:
# Tasks with different product types
tasks_with_types = [
    ('task1', 10, 0),  # TypeA, duration 10
    ('task2', 8, 1),   # TypeB, duration 8
    ('task3', 12, 0),  # TypeA, duration 12
    ('task4', 6, 2),   # TypeC, duration 6
    ('task5', 9, 1),   # TypeB, duration 9
]

# Create interval variables
task_intervals = []
task_types = []
for name, duration, product_type in tasks_with_types:
    iv = IntervalVar(size=duration, name=name)
    task_intervals.append(iv)
    task_types.append(product_type)
    type_name = ['TypeA', 'TypeB', 'TypeC'][product_type]
    print(f"{name}: duration={duration}, type={type_name}")

# Create sequence with type information
machine_seq = SequenceVar(
    intervals=task_intervals,
    types=task_types,  # Associate types with intervals
    name="machine_with_setup"
)

# No overlap with setup times
no_overlap_with_setup = SeqNoOverlap(machine_seq, transitions=setup_times)
print(f"\nSequence constraint with setup times: {no_overlap_with_setup}")

## Tutorial 5: Multi-Resource Scheduling

**Problem**: Tasks require multiple resources simultaneously.

**Key concept**: Model each resource as a cumulative function.

In [ ]:
clear_interval_registry()

# Resources: workers, machines, budget
resource_capacities = {
    'workers': 5,
    'machines': 2,
    'budget': 100,
}

# Tasks: (duration, workers_needed, machines_needed, budget_needed)
multi_resource_tasks = [
    ('build', 10, 3, 1, 50),
    ('test', 5, 2, 1, 20),
    ('deploy', 3, 1, 0, 30),
    ('review', 4, 2, 0, 10),
    ('analyze', 6, 1, 1, 40),
]

# Create intervals
intervals = {}
for name, duration, workers, machines, budget in multi_resource_tasks:
    intervals[name] = IntervalVar(size=duration, name=name)
    print(f"{name}: dur={duration}, workers={workers}, machines={machines}, budget={budget}")

In [ ]:
# Build cumulative functions for each resource
worker_usage = None
machine_usage = None
budget_usage = None

for name, duration, workers, machines, budget in multi_resource_tasks:
    iv = intervals[name]
    
    if workers > 0:
        p = pulse(iv, workers)
        worker_usage = p if worker_usage is None else worker_usage + p
    
    if machines > 0:
        p = pulse(iv, machines)
        machine_usage = p if machine_usage is None else machine_usage + p
    
    if budget > 0:
        p = pulse(iv, budget)
        budget_usage = p if budget_usage is None else budget_usage + p

# Constrain each resource
constraints = []
if worker_usage:
    c = cumul_range(worker_usage, 0, resource_capacities['workers'])
    constraints.append(c)
    print(f"Worker constraint: max {resource_capacities['workers']}")

if machine_usage:
    c = cumul_range(machine_usage, 0, resource_capacities['machines'])
    constraints.append(c)
    print(f"Machine constraint: max {resource_capacities['machines']}")

if budget_usage:
    c = cumul_range(budget_usage, 0, resource_capacities['budget'])
    constraints.append(c)
    print(f"Budget constraint: max {resource_capacities['budget']}")

print(f"\nTotal resource constraints: {len(constraints)}")

## Tutorial 6: Spanning and Synchronization

**Problem**: Model complex task hierarchies with span and synchronize constraints.

**Use case**: A project phase spans multiple subtasks; team members must synchronize for meetings.

In [ ]:
clear_interval_registry()

# Project structure:
# Project
# ├── Phase1 (spans design + implement)
# │   ├── Design
# │   └── Implement
# ├── Phase2 (spans test + deploy)
# │   ├── Test
# │   └── Deploy
# └── Kickoff Meeting (all team members synchronize)

# Create leaf tasks
design = IntervalVar(size=5, name="design")
implement = IntervalVar(size=10, name="implement")
test = IntervalVar(size=4, name="test")
deploy = IntervalVar(size=2, name="deploy")

# Create phase containers
phase1 = IntervalVar(name="phase1")
phase2 = IntervalVar(name="phase2")

# Span constraints: phases span their subtasks
span_phase1 = span(phase1, [design, implement])
span_phase2 = span(phase2, [test, deploy])

print("Phase1 spans: design + implement")
print("Phase2 spans: test + deploy")

# Precedence: phase1 before phase2
phase_order = end_before_start(phase1, phase2)
print("\nPhase1 must complete before Phase2 starts")

In [ ]:
# Team meeting: all attendees must synchronize
meeting = IntervalVar(size=60, name="kickoff_meeting")

# Team members (optional - they attend if present)
alice = IntervalVar(optional=True, name="alice_attends")
bob = IntervalVar(optional=True, name="bob_attends")
carol = IntervalVar(optional=True, name="carol_attends")

# All attendees synchronize with the meeting
sync_alice = synchronize(meeting, [alice])
sync_bob = synchronize(meeting, [bob])
sync_carol = synchronize(meeting, [carol])

print("Kickoff meeting: 60 time units")
print("All attendees synchronized with meeting time")
print(f"\nSynchronize constraints created:")
print(f"  - Alice sync: {sync_alice}")
print(f"  - Bob sync: {sync_bob}")
print(f"  - Carol sync: {sync_carol}")

---

## Summary

In this tutorial, you learned:

### Core Concepts
- **IntervalVar**: Represents tasks with start, end, and duration
- **SequenceVar**: Ordered collection of intervals on a resource
- **Optional intervals**: Tasks that may or may not be scheduled

### Constraint Types
| Category | Constraints |
|----------|-------------|
| **Precedence** | `end_before_start()`, `start_before_start()`, etc. |
| **Grouping** | `span()`, `alternative()`, `synchronize()` |
| **Sequence** | `SeqNoOverlap()`, `first()`, `last()`, `before()` |
| **Cumulative** | `pulse()`, `step_at_*()`, `cumul_range()` |
| **State** | `StateFunction`, `always_equal()`, `always_constant()` |

### Problem Patterns
1. **Job Shop**: Precedence + NoOverlap on machines
2. **RCPSP**: Precedence + Cumulative resource constraints
3. **Flexible Job Shop**: Alternative constraints for machine choice
4. **Setup Times**: TransitionMatrix with SeqNoOverlap
5. **Multi-Resource**: Multiple cumulative functions

### Next Steps
- Explore the [full API documentation](../docs/api/)
- Try the [example models](../examples/)
- Read the [modeling guide](../docs/user_guide/modeling_guide.md)